**извлечение признаков из текста на естественном языке**

класстеризатор текстов

частотный анализ с очисткой стоп-слов (TF)

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
1674,"Экс-президент СССР Михаил Горбачев заявил, что в границах Союза может появиться новое объединение, в которое добровольно войдут бывшие советские республики.\n\nОн отметил, что в качестве возврата ...",politics
2949,"Новое поколение Q3 копания Audi строит, что вполне ожидаемо, на модульной платформе MQB концерна Volkswagen. Как утверждает издание Auto Bild, в Ингольштадте собираются воспользоваться всеми преим...",auto


## токенизация и очистка

In [6]:
from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [7]:
stopwords = [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 
    'так', 'его', 'но',  'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 
    'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 
    'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или',  'ни', 'быть', 'был', 'него', 'до', 
    'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 
    'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем',
    'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 
    'ж', 'тогда', 'кто','этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 
    'этом', 'один', 'почти', 'мой', 'тем','чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем',
    'всех', 'никогда', 'можно', 'при', 'наконец', 'два','об', 'другой', 'хоть', 'после', 
    'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них','какая', 'много', 
    'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда',
    'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 
    'всегда', 'конечно', 'всю', 'между']

In [8]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ t for t in tt if t not in stopwords ] # удаление (предлогов)
    tt = Stemmer('russian').stemWords( tt ) # стемминг, выделение основы слова
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    tt = [ t for t in tt if len(t)>2 ] # удаление коротких слов (предлогов)
    return ' '.join( [ t.strip() for t in tt if t ] )
    

##  CountVectorizer + TF

In [47]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

# ручная очистка
# tf = TfidfVectorizer(preprocessor=preprocessor,use_idf=False,norm=None)
tf = TfidfVectorizer(preprocessor=preprocessor,use_idf=False,norm='l2')

tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7f1ea8635ea0>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=False, vocabulary=None)

In [48]:
# размер словаря
len(tf.vocabulary_)

37614

## формируем датасеты

In [49]:
X = tf.transform( data['text'] ) # .todense()
X.shape

(3196, 37614)

## кластеризируем

In [50]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [57]:
%%time

n_clusters = len(set(data['tag']))
# n_clusters = 5
clust = KMeans(n_clusters=n_clusters, random_state=get_seed() )
clust.fit(X)

CPU times: user 1min 8s, sys: 99.5 ms, total: 1min 8s
Wall time: 57.8 s


In [58]:
# set(clust.labels_)

In [59]:
data['cluster'] = clust.labels_

## тестируем

In [60]:
# размеры кластеров
[ (data['cluster']==c).sum() for c in range(n_clusters) ]

[133, 145, 47, 134, 313, 130, 129, 132, 46, 580, 641, 33, 733]

In [61]:
data[ data['cluster']==1 ].sample(10)

,text,tag,cluster
580,"В Поставском районе столкнулись грузовик и Renault: пострадали два человека\n\n1 декабря 2016 в 10:47\n\nAUTO.TUT.BY\n\nДва человека получили травмы различной степени тяжести в ДТП, которое произо...",incident,1
558,"Отец и дочь погибли в лобовой аварии в Дятловском районе, еще двое — в больнице\n\n3 декабря 2016 в 16:56\n\nAUTO.TUT.BY\n\nДва человека погибли, еще два получили травмы в ДТП, которое произошло 2...",incident,1
563,В Витебске лоб в лоб столкнулись милицейский автомобиль и медпомощь\n\n2 декабря 2016 в 15:50\n\nAUTO.TUT.BY\n\nДТП произошло 2 декабря в первом часу дня возле Кировского моста в Витебске. Столкну...,incident,1
2353,Все обстоятельства произошедшего будут установлены в ходе проверки. Фото: Иван ТИМОФЕЕВ\n\nВ Первомайском районе Ростова следователи проводят проверку по факту смерти семьи из трех человек. Тела о...,incident,1
2384,"Момент, когда внедорожник ""Гелендваген"" сбил детей на переходе на Мосфильмовской улице в столице, попал на видео. Лайф публикует кадры произошедшей аварии.\n\nНапомним, сегодня около половины девя...",incident,1
2230,"Авария произошла у дома 8 на проспекте Солидарности в Невском районе Петербурга. Грузовик Mitsubishi столкнулся с рейсовым автобусом №12, после чего автобус выехал на трамвайные рельсы. На место Д...",incident,1
2345,"Сотрудники правоохранительных органов в частном доме в Ростове-на-Дону нашли тела трех человек, - сообщил источник в силовых структурах региона.\n\n""Все трое погибших - взрослые люди. Предположите...",incident,1
2334,В Ростовской области сотрудники полиции устанавливают обстоятельства лобового столкновения двух грузовиков Volvo. В результате аварии травмы различной степени тяжести получили водители грузовых ма...,incident,1
2324,"МОСКВА, 13 декабря. /ТАСС/. Бывшая сотрудница полиции, сбившая в состоянии алкогольного опьянения двух пешеходов в подмосковном Серпухове, частично признала вину. Об этом сообщили ТАСС в пресс-слу...",incident,1
2273,"Пользователи Сети опубликовали видео с места массовой аварии на трассе в Кемеровской области. На кадрах видно, что на месте происшествия работают отряды спасателей, передаёт РЕН ТВ.\n\nМногие авто...",incident,1


In [37]:
# data[ data['cluster']==1 ].sample(10)

In [36]:
# data[ data['cluster']==2 ].sample(10)

In [35]:
# data[ data['cluster']==4 ].sample(10)

In [25]:
# data[ data['cluster']==9 ].sample(10)